In [1]:
#@title install packages
!pip install datasets transformers  accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [2]:
#@title add api keys
#!huggingface-cli login --token read_token
#!huggingface-cli login --token write_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
#@title Load and clean dataset
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("mor40/chitanka_raw_document")
dataset = dataset["train"]

def remove_tab(example):
    example["text"] =example["text"].replace('\t', '')
    example["text"] =example["text"].replace('_', '')
    return example

dataset = dataset.map(remove_tab, num_proc=4 )
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9910 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/9910 [00:00<?, ? examples/s]

## Train tokenizer

In [ ]:
from transformers import AutoTokenizer

# Get bert tokenizer
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",use_fast=True)

# get 500 examples of the dataset to train the tokenizer
tok_dataset = dataset.select(range(500))["text"]


# Making the iterator for the dataset
def get_training_corpus(dataset):
  for start_idx in tqdm(range(0, len(dataset),10)):
    yield dataset[start_idx: start_idx +10]

# create the training corpus as iterator
training_corpus = get_training_corpus(tok_dataset)

# train the new tokenizer from the old one (takes a long time)
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus,50265)

# save it
new_tokenizer.save_pretrained("BulBERT-chitanka")

# push to the huggingface hub
#new_tokenizer.push_to_hub("BulBERT-chitanka")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

100%|██████████| 50/50 [00:28<00:00,  1.75it/s]


## Tokenize the dataset

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mor40/BulBERT-chitanka")

def tokenize_function(examples):
    result = tokenizer(examples["text"])
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text"],
    batch_size=200,
)

In [ ]:
#@title Split the text in batches of 256 tokens
chunk_size = 256
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size

    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }

    # Create a new labels column
    result["labels"] = result["input_ids"].copy()

    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=200)
#lm_datasets.push_to_hub("mor40/tokenized_chitanka")

## Train the model

In [ ]:
#@title load tokenized dataset
dataset = load_dataset("mor40/tokenized_chitanka")
tokenizer = AutoTokenizer.from_pretrained('BulBERT-chitanka')

dataset

In [ ]:
# Test if tokenizer is right
tokenizer.decode(dataset["train"][112]["labels"])

In [ ]:
#@title Define model
from transformers import BertConfig , BertForPreTraining, BertForMaskedLM
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

config = BertConfig(
    vocab_size=50265,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
)

from transformers import BertForMaskedLM

# use this if you want to continue training
model = BertForMaskedLM.from_pretrained("mor40/BulBERT-chitanka-model")

# use this for training from scratch
#model = BertForMaskedLM(config=config)
print(model.num_parameters())

# Check that PyTorch sees it
import torch
print("Torch : ",torch.cuda.is_available())


from transformers import TrainingArguments


# Show the training loss with every epoch
logging_steps = 100
model_checkpoint = "mor40/BulBERT-chitanka-model"
#model_name = model_checkpoint.split("/")[-1]

batch_size = 32
# Show the training loss with every epoch

training_args = TrainingArguments(
    output_dir="mor40/BulBERT-chitanka-model",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    logging_steps=500,
    fp16=True
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"].select(range(500)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

import math

eval_results = trainer.evaluate()
print(f">>> Perplexity before training: {math.exp(eval_results['eval_loss']):.2f}")

trainer.train()


eval_results = trainer.evaluate()
print(f">>> Perplexity after training: {math.exp(eval_results['eval_loss']):.2f}")

## Test the model

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="mor40/BulBERT-chitanka-model")

In [ ]:
pipe("Странно е, че читателят спира да [MASK] всички части на книгата. Обикновено характеристиките на дизайна на този ценен ресурс остават [MASK].")